In [3]:
print(pd.__version__)

1.4.2


In [12]:
dict_bow = load('dict_bow.pkl')
data_for_tfidf = load('data_for_tfidf.pkl')
dict_tag = load('dict_tag_100.pkl')

In [ ]:
for i in range(2003):
    print(data_for_tfidf.get(i))

In [30]:
title_in = "How do I make a flat list out of a list of lists"
post_in = "Explanation: the shortcuts based on + (including the implied use in sum) are, of necessity, O(L**2) when there are L sublists -- as the intermediate result list keeps getting longer, at each step a new intermediate result list object gets allocated, and all the items in the previous intermediate result must be copied over (as well as a few new ones added at the end). So, for simplicity and without actual loss of generality, say you have L sublists of I items each: the first I items are copied back and forth L-1 times, the second I items L-2 times, and so on; total number of copies is I times the sum of x for x from 1 to L excluded, i.e., I * (L**2)/2"

In [19]:
def suppr_balises_html(text):
    soup = BeautifulSoup(text, "html.parser")
    for data in soup(['style', 'script']):
        data.decompose()

    text_out = ' '.join(soup.stripped_strings)
    return text_out

def tokenize_text(text):
    words = list(gensim.utils.tokenize(text, lowercase=True))
    return words

def suppr_stopwords(list):
    mystopwords = set(stopwords.words('english'))
    words = [x for x in list if x not in mystopwords and len(x) > 1]
    return words

def lemmatize(list):
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(x) for x in list]
    return words

In [31]:
title_body_api = pd.DataFrame()
post_out = suppr_balises_html(post_in)
title_out = tokenize_text(title_in)
post_out = tokenize_text(post_out)
title_out = suppr_stopwords(title_out)
post_out = suppr_stopwords(post_out)
title_out = lemmatize(title_out)
post_out = lemmatize(post_out)
title_body_api['Title_body'] = [title_out + post_out]

In [52]:
texts = title_body_api['Title_body']
texts = pd.concat([texts, data_for_tfidf['Title_body']])
bow_corpus = [dict_bow.doc2bow(text) for text in texts]

In [61]:
texts = title_body_api['Title_body']
texts

0    [make, flat, list, list, list, explanation, sh...
Name: Title_body, dtype: object

In [63]:
test = [post_out]

In [64]:
bow_corpus = [dict_bow.doc2bow(text) for text in test]

In [37]:
title_body_api['Title_body']

0    [make, flat, list, list, list, explanation, sh...
Name: Title_body, dtype: object

In [51]:
data_for_tfidf['Title_body']

38094    [problem, converting, matrix, data, frame, thi...
40624    [launch, batch, file, stored, different, direc...
49425    [making, function, template, specialization, v...
35734    [need, modify, jquery, cycle, updateactivepage...
41708    [equivalent, linux, free, command, freebsd, eq...
                               ...                        
48026    [valueerror, classification, metric, handle, m...
23855    [jersey, jax, r, glassfish, empty, webapp, ele...
19842    [puppeteer, get, iframe, parent, element, sele...
36517    [haskell, really, purely, functional, language...
41527    [browse, active, directory, silverlight, silve...
Name: Title_body, Length: 1000, dtype: object

In [69]:
texts = pd.concat([texts, data_for_tfidf['Title_body']])
bow_corpus = [dict_bow.doc2bow(text) for text in data_for_tfidf['Title_body']]

In [74]:
tfidf = TfidfModel(bow_corpus)
bow_tv_ft_ttb = [tfidf[text] for text in bow_corpus]
bow_tv_ft_ttb_test = [bow_tv_ft_ttb[0]]


In [ ]:
    X_test = gensim.matutils.corpus2csc(bow_tv_ft_ttb_test, num_terms=len(dict_bow))
    X_test = X_test.T.toarray()
    scaler = load('standardscaler')
    X_test_std = scaler.transform(X_test)
    clf = load('modele_reg_log')
    y_pred_test_proba = clf.predict_proba(X_test_std)

In [81]:
y_pred_test_proba_w = y_pred_test_proba.copy()
len(y_pred_test_proba_w)

1

In [97]:
    tag_predit = pd.DataFrame()
    for k in range(len(y_pred_test_proba_w)):
        for i in range(5):
            val_max = 0
            ind = -1
            for j in range(100):
                if y_pred_test_proba_w[k, j] > val_max:
                    val_max = y_pred_test_proba_w[k, j]
                    ind = j
            if ind > -1:
                tag_predit.loc[k, i] = dict_tag[ind]
                y_pred_test_proba_w[k, ind] = 0
    tag_predit['List_tags'] = tag_predit[0] + ' ' + tag_predit[1] + \
                              ' ' + tag_predit[2] + ' ' + tag_predit[3] + ' ' + tag_predit[4]
    Tags_out = tag_predit.loc[0]['List_tags']
    Tags_out

'gcc selenium web-services wordpress http'

In [84]:
for k in range(len(y_pred_test_proba_w)):
    print(k)

0


In [102]:
for i in range(5):
            val_max = 0
            ind = -1
            for j in range(100):
                if y_pred_test_proba_w[k, j] > val_max:
                    val_max = y_pred_test_proba_w[k, j]
                    ind = j
            if ind > -1:
                tag_predit.loc[k, i] = dict_tag[ind]
                y_pred_test_proba_w[k, ind] = 0
tag_predit['List_tags'] = tag_predit[0] + ' ' + tag_predit[1] + \
                              ' ' + tag_predit[2] + ' ' + tag_predit[3] + ' ' + tag_predit[4]
Tags_out = tag_predit.loc[0]['List_tags']
Tags_out

'optimization apache xcode bash iphone'